In [269]:
import pandas as pd
import numpy as np
from ast import literal_eval

## Orders

In [270]:
orders_df = pd.read_csv('data/train_orders.csv', index_col=0)

In [311]:
val_df = pd.read_csv('data/orders_val.csv', index_col=0)

In [333]:
test_df = pd.read_csv('data/orders_test.csv', index_col=0)

In [331]:
val_df.shape

(812883, 9)

In [332]:
orders_df.shape

(3458398, 9)

In [334]:
test_df.shape

(533544, 7)

In [271]:
orders_df.head()

,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target
0,user_1,1,1634292444,NaN,NaN,NaN,NaN,0
1,user_1,2,1639827297,NaN,NaN,NaN,NaN,0
2,user_1,2,1639827681,NaN,NaN,NaN,NaN,0
3,user_3,21,1684248081,NaN,NaN,NaN,NaN,1
4,user_3,22,1684892551,item_1,1.0,NaN,NaN,1


In [335]:
test_df.head()

,user-id,site-id,created-at,item-id,count,general-category-path,brand-id
0,user_155204,489,1405923052,NaN,NaN,NaN,NaN
1,user_155206,62,1621174041,item_13288,1.0,"[91214, 91197, 90801]",NaN
2,user_155206,62,1621174041,item_13288,1.0,"[91214, 91197, 90801]",NaN
3,user_155206,62,1670141242,item_6173525,1.0,NaN,NaN
4,user_155206,62,1670141242,item_6173525,1.0,NaN,NaN


Константы

In [272]:
# Граница отсечения результатов (настраивается в процессе тестов)
BORDER = 10

### Группировки

Разобьем список категория на отдельные строки

In [273]:
orders_df_with_category = orders_df[~orders_df['general-category-path'].isna()]
orders_df_with_category['general-category-path'] = orders_df_with_category['general-category-path'].apply(literal_eval)
orders_df_with_category = orders_df_with_category.explode('general-category-path')
orders_df_with_category

C:\Users\Demion\AppData\Local\Temp\ipykernel_12608\18327562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df_with_category['general-category-path'] = orders_df_with_category['general-category-path'].apply(literal_eval)


,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target
6,user_4,29,1611987662,item_9,1.0,90404,3243.0,1
6,user_4,29,1611987662,item_9,1.0,90403,3243.0,1
6,user_4,29,1611987662,item_9,1.0,90402,3243.0,1
7,user_4,29,1655426430,item_10,1.0,90698,1210537.0,1
7,user_4,29,1655426430,item_10,1.0,7330336,1210537.0,1
...,...,...,...,...,...,...,...,...
3458394,user_127755,58,1690265390,item_1896437,1.0,90813,14019.0,0
3458395,user_127755,58,1690265390,item_3739266,1.0,12718223,32654.0,0
3458395,user_127755,58,1690265390,item_3739266,1.0,90813,32654.0,0
3458396,user_127755,58,1690265390,item_57560,1.0,12718223,5783.0,0


In [313]:
val_df_with_category = val_df[~val_df['general-category-path'].isna()]
val_df_with_category['general-category-path'] = val_df_with_category['general-category-path'].apply(literal_eval)
val_df_with_category = val_df_with_category.explode('general-category-path')
val_df_with_category

C:\Users\Demion\AppData\Local\Temp\ipykernel_12608\2666386670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df_with_category['general-category-path'] = val_df_with_category['general-category-path'].apply(literal_eval)


,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target
1,user_127757,16,1636574129,item_3780263,2.0,90578,1237.0,1
1,user_127757,16,1636574129,item_3780263,2.0,90574,1237.0,1
1,user_127757,16,1636574129,item_3780263,2.0,198118,1237.0,1
2,user_127757,16,1637512604,item_5403882,2.0,90578,1237.0,1
2,user_127757,16,1637512604,item_5403882,2.0,90574,1237.0,1
...,...,...,...,...,...,...,...,...
812877,user_155202,53,1682604045,item_650359,1.0,8475840,13642.0,0
812881,user_155202,300,1688068054,item_650362,1.0,818863,915491.0,0
812881,user_155202,300,1688068054,item_650362,1.0,15068776,915491.0,0
812881,user_155202,300,1688068054,item_650362,1.0,7811881,915491.0,0


In [336]:
test_df_with_category = test_df[~test_df['general-category-path'].isna()]
test_df_with_category['general-category-path'] = test_df_with_category['general-category-path'].apply(literal_eval)
test_df_with_category = test_df_with_category.explode('general-category-path')
test_df_with_category

C:\Users\Demion\AppData\Local\Temp\ipykernel_12608\3275519986.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_with_category['general-category-path'] = test_df_with_category['general-category-path'].apply(literal_eval)


,user-id,site-id,created-at,item-id,count,general-category-path,brand-id
1,user_155206,62,1621174041,item_13288,1.0,91214,NaN
1,user_155206,62,1621174041,item_13288,1.0,91197,NaN
1,user_155206,62,1621174041,item_13288,1.0,90801,NaN
2,user_155206,62,1621174041,item_13288,1.0,91214,NaN
2,user_155206,62,1621174041,item_13288,1.0,91197,NaN
...,...,...,...,...,...,...,...
533540,user_173986,79,1563163857,item_6640459,1.0,7811873,1750.0
533540,user_173986,79,1563163857,item_6640459,1.0,7877999,1750.0
533541,user_173986,79,1563163857,item_6640460,1.0,7812151,1750.0
533541,user_173986,79,1563163857,item_6640460,1.0,7811877,1750.0


#### Сайт/категория

In [274]:
orders_df_with_category.groupby(by=['site-id', 'general-category-path']).agg(
    Count=('created-at', 'nunique'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)

,,Count,Gender
site-id,general-category-path,,
29,90407,10,1.0
20,91214,10,1.0
40,191211,11,1.0
139,91512,12,1.0
40,278423,10,1.0
...,...,...,...
439,12457383,17,0.0
93,91008,10,0.0
173,14222454,11,0.0


In [275]:
orders_site_category_count = orders_df_with_category.groupby(by=['site-id', 'general-category-path']).agg(
    Count=('created-at', 'nunique'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)['Gender'].to_dict()
orders_site_category_count

{(29, 90407): 1.0,
 (20, 91214): 1.0,
 (40, 191211): 1.0,
 (139, 91512): 1.0,
 (40, 278423): 1.0,
 (151, 14700323): 1.0,
 (348, 91052): 1.0,
 (1057, 91697): 1.0,
 (20, 14223099): 1.0,
 (151, 14382342): 1.0,
 (198, 91597): 1.0,
 (151, 15869579): 1.0,
 (20, 90486): 1.0,
 (20, 90469): 1.0,
 (25, 12333825): 1.0,
 (25, 10785222): 1.0,
 (25, 4951523): 1.0,
 (151, 14223319): 1.0,
 (20, 15694311): 1.0,
 (151, 14220244): 1.0,
 (198, 995914): 1.0,
 (20, 8231633): 1.0,
 (198, 90813): 1.0,
 (20, 13858512): 1.0,
 (830, 857707): 1.0,
 (41, 91461): 1.0,
 (348, 4165204): 1.0,
 (317, 278373): 1.0,
 (20, 91629): 1.0,
 (599, 14223539): 1.0,
 (29, 13792950): 1.0,
 (599, 91657): 1.0,
 (151, 15695136): 1.0,
 (148, 10834023): 1.0,
 (41, 1618974): 1.0,
 (20, 294648): 1.0,
 (41, 91649): 1.0,
 (151, 15092067): 1.0,
 (20, 469547): 1.0,
 (348, 10510546): 1.0,
 (186, 8264452): 1.0,
 (21, 14201356): 1.0,
 (186, 8268516): 1.0,
 (21, 14369211): 1.0,
 (830, 191211): 1.0,
 (198, 995923): 1.0,
 (20, 13199962): 1.0,
 (20

#### Бренд

In [276]:
orders_brand_count = orders_df.groupby(by=['brand-id'])['user-id', 'created-at', 'target'].agg(
    Count=('created-at', 'count'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)

C:\Users\Demion\AppData\Local\Temp\ipykernel_12608\248900050.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  orders_brand_count = orders_df.groupby(by=['brand-id'])['user-id', 'created-at', 'target'].agg(


In [277]:
orders_brand_count = orders_df.groupby(by=['brand-id'])['user-id', 'created-at', 'target'].agg(
    Count=('created-at', 'count'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)['Gender'].to_dict()

C:\Users\Demion\AppData\Local\Temp\ipykernel_12608\1649155032.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  orders_brand_count = orders_df.groupby(by=['brand-id'])['user-id', 'created-at', 'target'].agg(


In [278]:
orders_brand_count

{1618.0: 1.0,
 39928.0: 1.0,
 1933697.0: 1.0,
 1935148.0: 1.0,
 12357.0: 1.0,
 33997.0: 1.0,
 34096.0: 1.0,
 10618.0: 1.0,
 35116.0: 1.0,
 35730.0: 1.0,
 35799.0: 1.0,
 1196052.0: 1.0,
 37169.0: 1.0,
 8538.0: 1.0,
 37328.0: 1.0,
 37546.0: 1.0,
 6411.0: 1.0,
 2173.0: 1.0,
 13758.0: 1.0,
 31615.0: 1.0,
 18238.0: 1.0,
 1911917.0: 1.0,
 24249.0: 1.0,
 28212.0: 1.0,
 22876.0: 1.0,
 19101.0: 1.0,
 18344.0: 1.0,
 29992.0: 1.0,
 14550.0: 1.0,
 1944.0: 1.0,
 1903403.0: 1.0,
 2035.0: 1.0,
 30883.0: 1.0,
 1901582.0: 1.0,
 1920848.0: 1.0,
 6385.0: 1.0,
 9826.0: 1.0,
 1913908.0: 1.0,
 142178.0: 1.0,
 125035.0: 1.0,
 130238.0: 1.0,
 4723.0: 1.0,
 58908.0: 1.0,
 1208583.0: 1.0,
 135555.0: 1.0,
 1888053.0: 1.0,
 55737.0: 1.0,
 67017.0: 1.0,
 61348.0: 1.0,
 1203819.0: 1.0,
 640470.0: 1.0,
 3345.0: 1.0,
 953750.0: 1.0,
 41572.0: 1.0,
 1955195.0: 1.0,
 3449631.0: 1.0,
 6047.0: 1.0,
 6409.0: 0.9736842105263158,
 1898383.0: 0.967741935483871,
 17808.0: 0.9666666666666667,
 1971.0: 0.9655172413793104,
 1467

#### Сайт

In [279]:
orders_df.groupby(by=['site-id'])['user-id', 'created-at', 'target'].agg(
    Count=('created-at', 'count'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)

C:\Users\Demion\AppData\Local\Temp\ipykernel_12608\1879741261.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  orders_df.groupby(by=['site-id'])['user-id', 'created-at', 'target'].agg(


,Count,Gender
site-id,,
1395,17,1.0
694,43,1.0
1788,13,1.0
1648,11,1.0
1516,20,1.0
...,...,...
1298,11,0.0
1299,10,0.0
1338,20,0.0


In [280]:
orders_site_count = orders_df.groupby(by=['site-id'])['user-id', 'created-at', 'target'].agg(
    Count=('created-at', 'count'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)['Gender'].to_dict()

C:\Users\Demion\AppData\Local\Temp\ipykernel_12608\3218816917.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  orders_site_count = orders_df.groupby(by=['site-id'])['user-id', 'created-at', 'target'].agg(


In [281]:
orders_site_count

{1395: 1.0,
 694: 1.0,
 1788: 1.0,
 1648: 1.0,
 1516: 1.0,
 1473: 1.0,
 1472: 1.0,
 1419: 1.0,
 1361: 0.96,
 1372: 0.9523809523809523,
 1317: 0.95,
 772: 0.9230769230769231,
 1248: 0.9166666666666666,
 1237: 0.9166666666666666,
 599: 0.889763779527559,
 1167: 0.8873239436619719,
 713: 0.8780487804878049,
 1267: 0.8709677419354839,
 535: 0.8666666666666667,
 830: 0.8646616541353384,
 139: 0.8586666666666667,
 744: 0.85,
 488: 0.8490856125586664,
 1351: 0.8333333333333334,
 1403: 0.8235294117647058,
 855: 0.8157894736842105,
 848: 0.8125,
 273: 0.8032786885245902,
 1225: 0.8,
 198: 0.7969653179190751,
 153: 0.7951945080091534,
 1364: 0.7916666666666666,
 151: 0.7885220875870623,
 1057: 0.7870370370370371,
 297: 0.781021897810219,
 1392: 0.7777777777777778,
 270: 0.7777777777777778,
 283: 0.7744565217391305,
 705: 0.7727272727272727,
 1496: 0.7692307692307693,
 1356: 0.7659574468085106,
 231: 0.7647058823529411,
 23: 0.7634011090573013,
 102: 0.7592592592592593,
 383: 0.7582417582417582,


#### Сайт/бренд

In [282]:
orders_df.groupby(by=['site-id', 'brand-id']).agg(
    Count=('created-at', 'nunique'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)

,,Count,Gender
site-id,brand-id,,
148,1063.0,12,1.0
324,4670.0,10,1.0
187,6051.0,14,1.0
20,2031.0,29,1.0
151,30042.0,10,1.0
...,...,...,...
63,15723.0,10,0.0
135,1903503.0,10,0.0
58,3423547.0,12,0.0


In [283]:
orders_site_brand_count = orders_df.groupby(by=['site-id', 'brand-id']).agg(
    Count=('created-at', 'nunique'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)['Gender'].to_dict()

In [284]:
orders_site_brand_count

{(148, 1063.0): 1.0,
 (324, 4670.0): 1.0,
 (187, 6051.0): 1.0,
 (20, 2031.0): 1.0,
 (151, 30042.0): 1.0,
 (20, 125035.0): 1.0,
 (29, 2400.0): 1.0,
 (20, 3345.0): 1.0,
 (54, 33050.0): 1.0,
 (29, 1546.0): 1.0,
 (198, 58908.0): 1.0,
 (20, 18049.0): 1.0,
 (20, 1618.0): 1.0,
 (20, 1955195.0): 1.0,
 (40, 1901582.0): 1.0,
 (20, 2085.0): 1.0,
 (25, 1657.0): 1.0,
 (198, 3528.0): 1.0,
 (20, 23188.0): 1.0,
 (288, 119161.0): 1.0,
 (20, 6307.0): 1.0,
 (20, 6385.0): 1.0,
 (20, 34328.0): 1.0,
 (20, 29348.0): 1.0,
 (20, 26292.0): 1.0,
 (198, 14550.0): 1.0,
 (151, 5887.0): 1.0,
 (348, 3717.0): 1.0,
 (41, 2924.0): 1.0,
 (25, 6409.0): 1.0,
 (198, 2026.0): 1.0,
 (8, 22876.0): 1.0,
 (151, 5903.0): 1.0,
 (153, 10618.0): 1.0,
 (20, 29489.0): 1.0,
 (160, 1087.0): 1.0,
 (830, 1277.0): 1.0,
 (139, 1898383.0): 0.967741935483871,
 (20, 17808.0): 0.9629629629629629,
 (1167, 1864.0): 0.9615384615384616,
 (20, 40346.0): 0.9583333333333334,
 (20, 1166.0): 0.9583333333333334,
 (20, 1627.0): 0.9583333333333334,
 (112, 

#### Сайт/товар

In [285]:
orders_df.groupby(by=['site-id', 'item-id']).agg(
    Count=('created-at', 'nunique'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)

Count  Gender
site-id item-id                    
62      item_188149      12     1.0
16      item_394856      10     1.0
62      item_2112574     14     1.0
21      item_725308      11     1.0
29      item_1083407     11     1.0
...                     ...     ...
220     item_371273      11     0.0
        item_373481      14     0.0
        item_374141      11     0.0
84      item_266813      11     0.0
93      item_151083      10     0.0

[28597 rows x 2 columns]

In [286]:
orders_site_item_count = orders_df.groupby(by=['site-id', 'item-id']).agg(
    Count=('created-at', 'nunique'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)['Gender'].to_dict()

In [287]:
orders_site_item_count

{(62, 'item_188149'): 1.0,
 (16, 'item_394856'): 1.0,
 (62, 'item_2112574'): 1.0,
 (21, 'item_725308'): 1.0,
 (29, 'item_1083407'): 1.0,
 (97, 'item_121554'): 1.0,
 (62, 'item_2129365'): 1.0,
 (63, 'item_2480808'): 1.0,
 (11, 'item_1206969'): 1.0,
 (20, 'item_232790'): 1.0,
 (16, 'item_254460'): 1.0,
 (29, 'item_1025317'): 1.0,
 (147, 'item_695504'): 1.0,
 (49, 'item_171090'): 1.0,
 (20, 'item_229007'): 1.0,
 (62, 'item_4681413'): 1.0,
 (62, 'item_81903'): 1.0,
 (441, 'item_469021'): 1.0,
 (63, 'item_2479253'): 1.0,
 (16, 'item_424618'): 1.0,
 (488, 'item_23223'): 1.0,
 (488, 'item_529878'): 1.0,
 (119, 'item_913939'): 1.0,
 (48, 'item_157508'): 1.0,
 (20, 'item_56344'): 1.0,
 (16, 'item_7559'): 1.0,
 (507, 'item_1369210'): 1.0,
 (20, 'item_35915'): 1.0,
 (46, 'item_156266'): 1.0,
 (175, 'item_193576'): 1.0,
 (351, 'item_3969762'): 1.0,
 (351, 'item_2944261'): 1.0,
 (62, 'item_2109978'): 1.0,
 (348, 'item_145241'): 1.0,
 (79, 'item_572695'): 1.0,
 (20, 'item_871245'): 1.0,
 (20, 'item_

### Оценка

In [288]:
orders_df.head()

,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target
0,user_1,1,1634292444,NaN,NaN,NaN,NaN,0
1,user_1,2,1639827297,NaN,NaN,NaN,NaN,0
2,user_1,2,1639827681,NaN,NaN,NaN,NaN,0
3,user_3,21,1684248081,NaN,NaN,NaN,NaN,1
4,user_3,22,1684892551,item_1,1.0,NaN,NaN,1


In [316]:
orders_df['orders_id'] = orders_df.index
orders_df

,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target,orders_id
0,user_1,1,1634292444,NaN,NaN,NaN,NaN,0,0
1,user_1,2,1639827297,NaN,NaN,NaN,NaN,0,1
2,user_1,2,1639827681,NaN,NaN,NaN,NaN,0,2
3,user_3,21,1684248081,NaN,NaN,NaN,NaN,1,3
4,user_3,22,1684892551,item_1,1.0,NaN,NaN,1,4
...,...,...,...,...,...,...,...,...,...
3458393,user_127755,58,1690265390,item_162398,1.0,"[15685787, 12699910, 90813]",26642.0,0,3458393
3458394,user_127755,58,1690265390,item_1896437,1.0,"[4922657, 16087732, 90813]",14019.0,0,3458394
3458395,user_127755,58,1690265390,item_3739266,1.0,"[12718223, 90813]",32654.0,0,3458395
3458396,user_127755,58,1690265390,item_57560,1.0,"[12718223, 90813]",5783.0,0,3458396


In [315]:
val_df['orders_id'] = val_df.index
val_df

,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target,orders_id
0,user_127756,407,1697125567,item_676374,NaN,NaN,NaN,0,0
1,user_127757,16,1636574129,item_3780263,2.0,"[90578, 90574, 198118]",1237.0,1,1
2,user_127757,16,1637512604,item_5403882,2.0,"[90578, 90574, 198118]",1237.0,1,2
3,user_127757,16,1637948102,item_1627033,1.0,"[91498, 91497, 91461, 198119]",1.0,1,3
4,user_127757,16,1638631112,item_554781,1.0,"[90629, 90625, 10599873, 198119]",3519.0,1,4
...,...,...,...,...,...,...,...,...,...
812878,user_155202,53,1684604189,item_650360,1.0,NaN,NaN,0,812878
812879,user_155202,53,1684604189,item_650361,1.0,NaN,NaN,0,812879
812880,user_155202,54,1420434776,NaN,NaN,NaN,NaN,0,812880
812881,user_155202,300,1688068054,item_650362,1.0,"[818863, 15068776, 7811881, 7877999]",915491.0,0,812881


In [337]:
test_df['orders_id'] = test_df.index
test_df

,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,orders_id
0,user_155204,489,1405923052,NaN,NaN,NaN,NaN,0
1,user_155206,62,1621174041,item_13288,1.0,"[91214, 91197, 90801]",NaN,1
2,user_155206,62,1621174041,item_13288,1.0,"[91214, 91197, 90801]",NaN,2
3,user_155206,62,1670141242,item_6173525,1.0,NaN,NaN,3
4,user_155206,62,1670141242,item_6173525,1.0,NaN,NaN,4
...,...,...,...,...,...,...,...,...
533539,user_173986,79,1563163857,item_6640458,1.0,"[7811908, 7811873, 7877999]",1750.0,533539
533540,user_173986,79,1563163857,item_6640459,1.0,"[7811908, 7811873, 7877999]",1750.0,533540
533541,user_173986,79,1563163857,item_6640460,1.0,"[7812151, 7811877, 7877999]",1750.0,533541
533542,user_173986,79,1628941618,NaN,NaN,NaN,NaN,533542


site/category

In [291]:
site_category_orders = orders_df[~orders_df['general-category-path'].isna()][['user-id', 'orders_id', 'site-id', 'general-category-path']]
site_category_orders['category'] = site_category_orders[~site_category_orders['general-category-path'].isna()]['general-category-path'].apply(literal_eval)
site_category_orders = site_category_orders.drop('general-category-path', axis=1)
site_category_orders = site_category_orders.explode('category')
site_category_orders['site_category'] = list(zip(site_category_orders['site-id'], site_category_orders['category']))
site_category_orders = site_category_orders.drop(['site-id', 'category'], axis=1)
site_category_orders['site_category_p'] = site_category_orders['site_category'].apply(lambda x: orders_site_category_count.get(x, None))
site_category_orders = site_category_orders.drop(['site_category'], axis=1)
site_category_orders

In [317]:
site_category_orders_val = val_df[~val_df['general-category-path'].isna()][['user-id', 'orders_id', 'site-id', 'general-category-path']]
site_category_orders_val['category'] = site_category_orders_val[~site_category_orders_val['general-category-path'].isna()]['general-category-path'].apply(literal_eval)
site_category_orders_val = site_category_orders_val.drop('general-category-path', axis=1)
site_category_orders_val = site_category_orders_val.explode('category')
site_category_orders_val['site_category'] = list(zip(site_category_orders_val['site-id'], site_category_orders_val['category']))
site_category_orders_val = site_category_orders_val.drop(['site-id', 'category'], axis=1)
site_category_orders_val['site_category_p'] = site_category_orders_val['site_category'].apply(lambda x: orders_site_category_count.get(x, None))
site_category_orders_val = site_category_orders_val.drop(['site_category'], axis=1)
site_category_orders_val

,user-id,orders_id,site_category_p
1,user_127757,1,0.662338
1,user_127757,1,0.537074
1,user_127757,1,0.491353
2,user_127757,2,0.662338
2,user_127757,2,0.537074
...,...,...,...
812877,user_155202,812877,0.189825
812881,user_155202,812881,NaN
812881,user_155202,812881,NaN
812881,user_155202,812881,NaN


In [338]:
site_category_orders_test = test_df[~test_df['general-category-path'].isna()][['user-id', 'orders_id', 'site-id', 'general-category-path']]
site_category_orders_test['category'] = site_category_orders_test[~site_category_orders_test['general-category-path'].isna()]['general-category-path'].apply(literal_eval)
site_category_orders_test = site_category_orders_test.drop('general-category-path', axis=1)
site_category_orders_test = site_category_orders_test.explode('category')
site_category_orders_test['site_category'] = list(zip(site_category_orders_test['site-id'], site_category_orders_test['category']))
site_category_orders_test = site_category_orders_test.drop(['site-id', 'category'], axis=1)
site_category_orders_test['site_category_p'] = site_category_orders_test['site_category'].apply(lambda x: orders_site_category_count.get(x, None))
site_category_orders_test = site_category_orders_test.drop(['site_category'], axis=1)
site_category_orders_test

,user-id,orders_id,site_category_p
1,user_155206,1,0.347471
1,user_155206,1,0.334876
1,user_155206,1,0.329917
2,user_155206,2,0.347471
2,user_155206,2,0.334876
...,...,...,...
533540,user_173986,533540,0.518414
533540,user_173986,533540,0.609076
533541,user_173986,533541,0.607491
533541,user_173986,533541,0.619167


first category

In [294]:
site_category_orders_first = site_category_orders.dropna().drop_duplicates(subset = ['user-id', 'orders_id'])
site_category_orders_first = site_category_orders_first.groupby(by=['user-id', 'orders_id'], as_index=False).agg(
    site_category_p = ('site_category_p', 'mean'))
site_category_orders_first = site_category_orders_first.groupby(by='user-id', as_index=False).agg(
    site_category_p = ('site_category_p', 'mean'))
site_category_orders_first

,user-id,site_category_p
0,user_1000,0.436521
1,user_100000,0.553529
2,user_100001,0.564460
3,user_100002,0.254429
4,user_100003,0.554622
...,...,...
77949,user_99992,0.717744
77950,user_99993,0.247570
77951,user_99994,0.831483
77952,user_99995,0.206823


In [319]:
site_category_orders_first_val = site_category_orders_val.dropna().drop_duplicates(subset = ['user-id', 'orders_id'])
site_category_orders_first_val = site_category_orders_first_val.groupby(by=['user-id', 'orders_id'], as_index=False).agg(
    site_category_p = ('site_category_p', 'mean'))
site_category_orders_first_val = site_category_orders_first_val.groupby(by='user-id', as_index=False).agg(
    site_category_p = ('site_category_p', 'mean'))
site_category_orders_first_val

,user-id,site_category_p
0,user_127757,0.613322
1,user_127758,0.332267
2,user_127759,0.148793
3,user_127761,0.417320
4,user_127763,0.579524
...,...,...
16971,user_155196,0.265991
16972,user_155197,0.532962
16973,user_155198,0.379480
16974,user_155200,0.296504


In [339]:
site_category_orders_first_test = site_category_orders_test.dropna().drop_duplicates(subset = ['user-id', 'orders_id'])
site_category_orders_first_test = site_category_orders_first_test.groupby(by=['user-id', 'orders_id'], as_index=False).agg(
    site_category_p = ('site_category_p', 'mean'))
site_category_orders_first_test = site_category_orders_first_test.groupby(by='user-id', as_index=False).agg(
    site_category_p = ('site_category_p', 'mean'))
site_category_orders_first_test

,user-id,site_category_p
0,user_155206,0.428609
1,user_155207,0.181522
2,user_155208,0.349959
3,user_155210,0.198714
4,user_155211,0.536765
...,...,...
11529,user_173982,0.549530
11530,user_173983,0.118441
11531,user_173984,0.229629
11532,user_173985,0.501887


last category

In [295]:
site_category_orders_last = site_category_orders.drop_duplicates(subset = ['user-id', 'orders_id'], keep='last')
site_category_orders_last = site_category_orders_last.groupby(by=['user-id', 'orders_id'], as_index=False).agg(
    site_category_p = ('site_category_p', 'mean'))
site_category_orders_last = site_category_orders_last.groupby(by='user-id', as_index=False).agg(
    site_category_p = ('site_category_p', 'mean'))
site_category_orders_last

,user-id,site_category_p
0,user_1000,0.431783
1,user_100000,0.542966
2,user_100001,0.584327
3,user_100002,0.204271
4,user_100003,0.535086
...,...,...
78036,user_99992,0.614769
78037,user_99993,0.278378
78038,user_99994,0.787284
78039,user_99995,0.220515


При использовании первой категории ошибок меньше

In [296]:
site_category_orders = site_category_orders.drop_duplicates(subset = ['user-id', 'orders_id'], keep='last')
site_category_orders = site_category_orders.groupby(by=['user-id', 'orders_id'], as_index=False).agg(
    site_category_p = ('site_category_p', 'mean'))

In [297]:
test_df1 = site_category_orders_first.merge(orders_df[['user-id', 'target']], how='left', on='user-id').drop_duplicates(subset=['user-id', 'site_category_p'])
test_df1['p'] = test_df1['site_category_p'].apply(lambda x: 1 if x >= 0.5 else 0)
test_df1['res'] = abs(test_df1['p'] - test_df1['target'])
test_df1['res'].mean()

0.2734946250352772

In [298]:
test_df1 = site_category_orders_last.merge(orders_df[['user-id', 'target']], how='left', on='user-id').drop_duplicates(subset=['user-id', 'site_category_p'])
test_df1['p'] = test_df1['site_category_p'].apply(lambda x: 1 if x >= 0.5 else 0)
test_df1['res'] = abs(test_df1['p'] - test_df1['target'])
test_df1['res'].mean()

0.3007009136223267

item/orders

In [299]:
site_item_orders = orders_df[['user-id', 'orders_id', 'site-id', 'item-id']].dropna()
site_item_orders['site_item'] = list(zip(site_item_orders['site-id'], site_item_orders['item-id']))
site_item_orders = site_item_orders.drop(['site-id', 'item-id'], axis=1)
site_item_orders['site_item_p'] = site_item_orders['site_item'].apply(lambda x: orders_site_item_count.get(x, None))
site_item_orders = site_item_orders.drop(['site_item'], axis=1)
site_item_orders

,user-id,orders_id,site_item_p
4,user_3,4,NaN
5,user_4,5,0.556870
6,user_4,6,NaN
7,user_4,7,NaN
8,user_4,8,NaN
...,...,...,...
3458393,user_127755,3458393,NaN
3458394,user_127755,3458394,NaN
3458395,user_127755,3458395,NaN
3458396,user_127755,3458396,0.130435


In [320]:
site_item_orders_val = val_df[['user-id', 'orders_id', 'site-id', 'item-id']].dropna()
site_item_orders_val['site_item'] = list(zip(site_item_orders_val['site-id'], site_item_orders_val['item-id']))
site_item_orders_val = site_item_orders_val.drop(['site-id', 'item-id'], axis=1)
site_item_orders_val['site_item_p'] = site_item_orders_val['site_item'].apply(lambda x: orders_site_item_count.get(x, None))
site_item_orders_val = site_item_orders_val.drop(['site_item'], axis=1)
site_item_orders_val

,user-id,orders_id,site_item_p
0,user_127756,0,0.589744
1,user_127757,1,NaN
2,user_127757,2,NaN
3,user_127757,3,NaN
4,user_127757,4,NaN
...,...,...,...
812877,user_155202,812877,NaN
812878,user_155202,812878,NaN
812879,user_155202,812879,NaN
812881,user_155202,812881,NaN


In [340]:
site_item_orders_test = test_df[['user-id', 'orders_id', 'site-id', 'item-id']].dropna()
site_item_orders_test['site_item'] = list(zip(site_item_orders_test['site-id'], site_item_orders_test['item-id']))
site_item_orders_test = site_item_orders_test.drop(['site-id', 'item-id'], axis=1)
site_item_orders_test['site_item_p'] = site_item_orders_test['site_item'].apply(lambda x: orders_site_item_count.get(x, None))
site_item_orders_test = site_item_orders_test.drop(['site_item'], axis=1)
site_item_orders_test

,user-id,orders_id,site_item_p
1,user_155206,1,0.538462
2,user_155206,2,0.538462
3,user_155206,3,NaN
4,user_155206,4,NaN
5,user_155206,5,0.372549
...,...,...,...
533538,user_173986,533538,NaN
533539,user_173986,533539,NaN
533540,user_173986,533540,NaN
533541,user_173986,533541,NaN


site/brand

In [300]:
site_brand_orders = orders_df[['user-id', 'orders_id', 'site-id', 'brand-id']].dropna()
site_brand_orders['site_brand'] = list(zip(site_brand_orders['site-id'], site_brand_orders['brand-id']))
site_brand_orders = site_brand_orders.drop(['site-id', 'brand-id'], axis=1)
site_brand_orders['site_brand_p'] = site_brand_orders['site_brand'].apply(lambda x: orders_site_brand_count.get(x, None))
site_brand_orders = site_brand_orders.drop(['site_brand'], axis=1)
site_brand_orders

,user-id,orders_id,site_brand_p
6,user_4,6,0.727273
7,user_4,7,0.504132
8,user_4,8,0.518293
15,user_5,15,0.571429
16,user_5,16,0.488827
...,...,...,...
3458392,user_127755,3458392,0.175000
3458393,user_127755,3458393,0.175000
3458394,user_127755,3458394,0.179487
3458395,user_127755,3458395,0.067797


In [321]:
site_brand_orders_val = val_df[['user-id', 'orders_id', 'site-id', 'brand-id']].dropna()
site_brand_orders_val['site_brand'] = list(zip(site_brand_orders_val['site-id'], site_brand_orders_val['brand-id']))
site_brand_orders_val = site_brand_orders_val.drop(['site-id', 'brand-id'], axis=1)
site_brand_orders_val['site_brand_p'] = site_brand_orders_val['site_brand'].apply(lambda x: orders_site_brand_count.get(x, None))
site_brand_orders_val = site_brand_orders_val.drop(['site_brand'], axis=1)
site_brand_orders_val

,user-id,orders_id,site_brand_p
1,user_127757,1,0.650000
2,user_127757,2,0.650000
3,user_127757,3,0.576923
4,user_127757,4,0.647059
5,user_127757,5,0.519573
...,...,...,...
812869,user_155202,812869,0.180328
812870,user_155202,812870,0.133333
812876,user_155202,812876,0.158798
812877,user_155202,812877,0.158798


In [341]:
site_brand_orders_test = test_df[['user-id', 'orders_id', 'site-id', 'brand-id']].dropna()
site_brand_orders_test['site_brand'] = list(zip(site_brand_orders_test['site-id'], site_brand_orders_test['brand-id']))
site_brand_orders_test = site_brand_orders_test.drop(['site-id', 'brand-id'], axis=1)
site_brand_orders_test['site_brand_p'] = site_brand_orders_test['site_brand'].apply(lambda x: orders_site_brand_count.get(x, None))
site_brand_orders_test = site_brand_orders_test.drop(['site_brand'], axis=1)
site_brand_orders_test

,user-id,orders_id,site_brand_p
5,user_155206,5,0.254525
6,user_155206,6,NaN
7,user_155206,7,NaN
8,user_155206,8,0.291667
9,user_155206,9,0.291667
...,...,...,...
533517,user_173986,533517,0.366667
533518,user_173986,533518,0.476744
533539,user_173986,533539,0.623495
533540,user_173986,533540,0.623495


brand

In [301]:
brand_orders = orders_df[['user-id', 'orders_id', 'brand-id']].dropna()
brand_orders['brand_p'] = brand_orders['brand-id'].apply(lambda x: orders_brand_count.get(x, None))
brand_orders = brand_orders.drop(['brand-id'], axis=1)
brand_orders

,user-id,orders_id,brand_p
6,user_4,6,0.794872
7,user_4,7,0.512000
8,user_4,8,0.580769
15,user_5,15,0.598895
16,user_5,16,0.507717
...,...,...,...
3458392,user_127755,3458392,0.176309
3458393,user_127755,3458393,0.176309
3458394,user_127755,3458394,0.110000
3458395,user_127755,3458395,0.098361


In [322]:
brand_orders_val = val_df[['user-id', 'orders_id', 'brand-id']].dropna()
brand_orders_val['brand_p'] = brand_orders_val['brand-id'].apply(lambda x: orders_brand_count.get(x, None))
brand_orders_val = brand_orders_val.drop(['brand-id'], axis=1)
brand_orders_val

,user-id,orders_id,brand_p
1,user_127757,1,0.573207
2,user_127757,2,0.573207
3,user_127757,3,0.598895
4,user_127757,4,0.666667
5,user_127757,5,0.519787
...,...,...,...
812869,user_155202,812869,0.198795
812870,user_155202,812870,0.129032
812876,user_155202,812876,0.158798
812877,user_155202,812877,0.158798


In [342]:
brand_orders_test = test_df[['user-id', 'orders_id', 'brand-id']].dropna()
brand_orders_test['brand_p'] = brand_orders_test['brand-id'].apply(lambda x: orders_brand_count.get(x, None))
brand_orders_test = brand_orders_test.drop(['brand-id'], axis=1)
brand_orders_test

,user-id,orders_id,brand_p
5,user_155206,5,0.254525
6,user_155206,6,NaN
7,user_155206,7,NaN
8,user_155206,8,0.395161
9,user_155206,9,0.395161
...,...,...,...
533517,user_173986,533517,0.475138
533518,user_173986,533518,0.499819
533539,user_173986,533539,0.623495
533540,user_173986,533540,0.623495


site

In [302]:
site_orders = orders_df[['user-id', 'orders_id', 'site-id']].dropna()
site_orders['site_p'] = site_orders['site-id'].apply(lambda x: orders_site_count.get(x, None))
site_orders = site_orders.drop(['site-id'], axis=1)
site_orders

,user-id,orders_id,site_p
0,user_1,0,0.603833
1,user_1,1,0.565698
2,user_1,2,0.565698
3,user_3,3,0.390117
4,user_3,4,0.463999
...,...,...,...
3458393,user_127755,3458393,0.183846
3458394,user_127755,3458394,0.183846
3458395,user_127755,3458395,0.183846
3458396,user_127755,3458396,0.183846


In [323]:
site_orders_val = val_df[['user-id', 'orders_id', 'site-id']].dropna()
site_orders_val['site_p'] = site_orders_val['site-id'].apply(lambda x: orders_site_count.get(x, None))
site_orders_val = site_orders_val.drop(['site-id'], axis=1)
site_orders_val

,user-id,orders_id,site_p
0,user_127756,0,0.242673
1,user_127757,1,0.556956
2,user_127757,2,0.556956
3,user_127757,3,0.556956
4,user_127757,4,0.556956
...,...,...,...
812878,user_155202,812878,0.178472
812879,user_155202,812879,0.178472
812880,user_155202,812880,0.351187
812881,user_155202,812881,0.059701


In [343]:
site_orders_test = test_df[['user-id', 'orders_id', 'site-id']].dropna()
site_orders_test['site_p'] = site_orders_test['site-id'].apply(lambda x: orders_site_count.get(x, None))
site_orders_test = site_orders_test.drop(['site-id'], axis=1)
site_orders_test

,user-id,orders_id,site_p
0,user_155204,0,0.501585
1,user_155206,1,0.374828
2,user_155206,2,0.374828
3,user_155206,3,0.374828
4,user_155206,4,0.374828
...,...,...,...
533539,user_173986,533539,0.603105
533540,user_173986,533540,0.603105
533541,user_173986,533541,0.603105
533542,user_173986,533542,0.603105


#### Слияние

train

In [304]:
result = site_item_orders.merge(site_brand_orders, how='left', on=['user-id', 'orders_id'])
result = result.merge(site_category_orders, how='left', on=['user-id', 'orders_id'])
result = result.merge(brand_orders, how='left', on=['user-id', 'orders_id'])
result = result.merge(site_orders, how='left', on=['user-id', 'orders_id'])

result = result.drop('orders_id', axis=1).groupby(by='user-id', as_index = False).mean()

In [306]:
result.merge(orders_df[['user-id', 'target']], on='user-id').drop_duplicates().head(50)

,user-id,site_item_p,site_brand_p,site_category_p,brand_p,site_p,target
0,user_100,0.539431,NaN,NaN,NaN,0.565698,1
3,user_1000,0.442024,0.260790,0.431783,0.420493,0.558870,1
24,user_100000,NaN,0.657492,0.542966,0.595133,0.501558,1
27,user_100001,0.267102,0.577912,0.584327,0.598895,0.255716,0
56,user_100002,0.429019,0.209927,0.204271,0.218735,0.323290,0
94,user_100003,0.666667,0.596178,0.535086,0.635333,0.533189,1
112,user_100004,0.155196,0.163676,0.182735,0.169356,0.185536,0
117,user_100005,NaN,0.222874,0.326664,0.182280,0.501558,0
124,user_100006,0.323592,0.370787,0.184196,0.370466,0.241668,1
131,user_100007,0.444789,0.582220,0.383678,0.497388,0.428995,1


In [307]:
result.merge(orders_df[['user-id', 'target']], on='user-id').drop_duplicates().to_csv('predict/test_1.csv')

val

In [324]:
result_val = site_item_orders_val.merge(site_brand_orders_val, how='left', on=['user-id', 'orders_id'])
result_val = result_val.merge(site_category_orders_val, how='left', on=['user-id', 'orders_id'])
result_val = result_val.merge(brand_orders_val, how='left', on=['user-id', 'orders_id'])
result_val = result_val.merge(site_orders_val, how='left', on=['user-id', 'orders_id'])

result_val = result_val.drop('orders_id', axis=1).groupby(by='user-id', as_index = False).mean()

In [326]:
result_val.merge(val_df[['user-id', 'target']], on='user-id').drop_duplicates().head(50)

,user-id,site_item_p,site_brand_p,site_category_p,brand_p,site_p,target
0,user_127756,0.589744,NaN,NaN,NaN,0.242673,0
1,user_127757,NaN,0.597405,0.584942,0.565219,0.583797,1
17,user_127758,0.228709,0.340759,0.333877,0.362997,0.377651,0
55,user_127759,0.144602,0.134961,0.139234,0.137373,0.133938,0
107,user_127761,0.587382,0.608333,0.399367,0.552459,0.452935,0
111,user_127762,0.469388,NaN,NaN,NaN,0.450739,0
120,user_127763,0.489238,0.580793,0.564188,0.609247,0.531097,1
126,user_127764,NaN,NaN,NaN,NaN,0.240218,1
129,user_127765,0.561965,0.631579,0.562568,0.614108,0.560971,0
138,user_127766,0.114366,0.269811,0.249995,0.240863,0.251571,0


In [329]:
result_val.merge(val_df[['user-id', 'target']], on='user-id').drop_duplicates().to_csv('predict/val.csv')

test

In [344]:
result_test = site_item_orders_test.merge(site_brand_orders_test, how='left', on=['user-id', 'orders_id'])
result_test = result_test.merge(site_category_orders_test, how='left', on=['user-id', 'orders_id'])
result_test = result_test.merge(brand_orders_test, how='left', on=['user-id', 'orders_id'])
result_test = result_test.merge(site_orders_test, how='left', on=['user-id', 'orders_id'])

result_test = result_test.drop('orders_id', axis=1).groupby(by='user-id', as_index = False).mean()

In [347]:
result_test.drop_duplicates().head(50)

,user-id,site_item_p,site_brand_p,site_category_p,brand_p,site_p
0,user_155206,0.538507,0.462303,0.439389,0.426700,0.480652
1,user_155207,0.114194,0.172770,0.196837,0.175147,0.128712
2,user_155208,0.356360,0.372149,0.376890,0.336166,0.414160
3,user_155209,NaN,NaN,NaN,NaN,0.463999
4,user_155210,0.217596,0.151250,0.206400,0.217269,0.196409
5,user_155211,NaN,0.559524,0.541802,0.573542,0.566774
6,user_155212,0.504711,0.220796,0.214103,0.223187,0.245136
7,user_155213,NaN,0.825452,0.713741,0.632860,0.551285
8,user_155214,0.240991,0.193066,0.214515,0.196392,0.239809
9,user_155215,NaN,0.416088,0.384278,0.471029,0.414544


In [348]:
result_test.drop_duplicates().to_csv('predict/test.csv')

In [308]:
pd.read_csv('predict/test_1.csv', index_col=0)

,user-id,site_item_p,site_brand_p,site_category_p,brand_p,site_p,target
0,user_100,0.539431,NaN,NaN,NaN,0.565698,1
3,user_1000,0.442024,0.260790,0.431783,0.420493,0.558870,1
24,user_100000,NaN,0.657492,0.542966,0.595133,0.501558,1
27,user_100001,0.267102,0.577912,0.584327,0.598895,0.255716,0
56,user_100002,0.429019,0.209927,0.204271,0.218735,0.323290,0
...,...,...,...,...,...,...,...
3429871,user_99993,0.153846,0.424779,0.278378,0.152861,0.356570,0
3429880,user_99994,0.799587,0.758791,0.787284,0.762178,0.626134,1
3429891,user_99995,NaN,0.226528,0.220515,0.226476,0.226039,0
3429892,user_99996,0.462798,0.496476,0.422215,0.359006,0.420606,1


In [330]:
pd.read_csv('predict/val.csv', index_col=0)

,user-id,site_item_p,site_brand_p,site_category_p,brand_p,site_p,target
0,user_127756,0.589744,NaN,NaN,NaN,0.242673,0
1,user_127757,NaN,0.597405,0.584942,0.565219,0.583797,1
17,user_127758,0.228709,0.340759,0.333877,0.362997,0.377651,0
55,user_127759,0.144602,0.134961,0.139234,0.137373,0.133938,0
107,user_127761,0.587382,0.608333,0.399367,0.552459,0.452935,0
...,...,...,...,...,...,...,...
806817,user_155196,0.473946,NaN,0.301113,NaN,0.409189,0
806828,user_155197,0.560514,0.533449,0.545743,0.551614,0.584392,1
806842,user_155198,0.562179,0.426543,0.366100,0.426340,0.354969,0
806851,user_155200,0.240335,0.213166,0.221401,0.176991,0.196568,0


In [349]:
pd.read_csv('predict/test.csv', index_col=0)

,user-id,site_item_p,site_brand_p,site_category_p,brand_p,site_p
0,user_155206,0.538507,0.462303,0.439389,0.426700,0.480652
1,user_155207,0.114194,0.172770,0.196837,0.175147,0.128712
2,user_155208,0.356360,0.372149,0.376890,0.336166,0.414160
3,user_155209,NaN,NaN,NaN,NaN,0.463999
4,user_155210,0.217596,0.151250,0.206400,0.217269,0.196409
...,...,...,...,...,...,...
15023,user_173982,NaN,0.522252,0.559265,0.552459,0.533189
15024,user_173983,0.318656,0.075949,0.142692,0.075949,0.255642
15025,user_173984,0.191760,0.302218,0.234247,0.305477,0.229774
15026,user_173985,0.349867,0.580247,0.473591,0.481964,0.377015
